# Check forcing of greenhouse gases

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from fair21.constants.gases import molwt, burden_per_emission, lifetime
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.constants.general import TIME_AXIS, SPECIES_AXIS
from fair21.defaults import gas_list, n_gas_boxes
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
)
from fair21.defaults.gases import radiative_efficiency
from fair21.gas_cycle.forward import step_concentration
from fair21.forcing.ghg import ghg

In [ ]:
scenarios = ['ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp434', 'ssp460', 'ssp534-over', 'ssp585']

In [ ]:
# grab some emissions
# we need to track indices of CO2, CH4 and N2O
gas_index_mapping = {}
emissions_array = np.ones((8, 43, 751, 1)) * np.nan
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(gas_list):
        gas_index_mapping[gas] = igas
        gas_rcmip_name = gas.replace("-", "")
        emissions_array[iscen, igas, ...] = df.loc[
            (df['Scenario']==scenario) & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
        ].interpolate(axis=1).values.T
    
        # CO2 and N2O units need to behave
        if gas in ('CO2', 'N2O'):
            emissions_array[iscen, igas, :, :] = emissions_array[iscen, igas, :, :] / 1000

In [ ]:
gas_index_mapping

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp585-mean-temperature.csv')
ssp585_temperature_rfmip = df['temperature'].values

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}
effective_radiative_forcing = {}

n_gases = len(gas_list)
n_timesteps = 751
n_scenarios = len(scenarios)
n_gases

In [ ]:
# initialise arrays. Using arrays makes things run quicker

# remember: scen, species, time, box
# where we don't care about keeping outputs we use a singleton dimension to maintain bookkeeping
# possible memory saver: differentiate between ensembles where we vary emissions and those where we vary climate parameters

# this scenario definition needs to be part of the initialisation, either from pyam, scmdata or fair emissions files
concentration_array = np.ones((n_scenarios, n_gases, n_timesteps, 1)) * np.nan
g0 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
g1 = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
alpha_lifetime = np.ones((n_scenarios, n_gases, 1, 1))
airborne_emissions = np.zeros((n_scenarios, n_gases, 1, 1))
gas_boxes = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
iirf_0_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_cumulative_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_temperature_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
iirf_airborne_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
burden_per_emission_array = np.ones((1, n_gases, 1, 1)) * np.nan
lifetime_array = np.ones((n_scenarios, n_gases, 1, n_gas_boxes)) * np.nan
pre_industrial_concentration_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
partition_fraction_array = np.zeros((n_scenarios, n_gases, 1, n_gas_boxes))
natural_emissions_adjustment_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan

cumulative_emissions_array = np.cumsum(emissions_array, axis=TIME_AXIS)

# this one for the forcing
tropospheric_adjustment_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan
radiative_efficiency_array = np.ones((n_scenarios, n_gases, 1, 1)) * np.nan

for igas, gas in enumerate(gas_list):
    lifetime_array[:, igas, :, :] = lifetime[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    iirf_0_array[:, igas, :, :] = iirf_0[gas]
    iirf_cumulative_array[:, igas, :, :] = iirf_cumulative[gas]
    iirf_temperature_array[:, igas, :, :] = iirf_temperature[gas]
    iirf_airborne_array[:, igas, :, :] = iirf_airborne[gas]
    burden_per_emission_array[:, igas, :, :] = burden_per_emission[gas]
    pre_industrial_concentration_array[:, igas, :, :] = pre_industrial_concentration[gas]
    partition_fraction_array[:, igas, :, :] = partition_fraction[gas]
    natural_emissions_adjustment_array[:, igas, :, :] = natural_emissions_adjustment[gas]
    
    # this one for the forcing
    tropospheric_adjustment_array[:, igas, :, :] = tropospheric_adjustment[gas]
    if gas not in ("CO2", "CH4", "N2O"):
        radiative_efficiency_array[:, igas, :, :] = radiative_efficiency[gas]

g0, g1 = calculate_g(lifetime_array, partition_fraction_array)

In [ ]:
for i_timestep in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions_array[:, :, [i_timestep], :],
        airborne_emissions[:, :, [0], :],
        ssp585_temperature_rfmip[i_timestep],
        iirf_0_array,
        iirf_cumulative_array,
        iirf_temperature_array,
        iirf_airborne_array,
        g0,
        g1,
    )
    alpha_lifetime[np.isnan(alpha_lifetime)]=1  # CF4 seems to have an issue. Should we raise warning?
    concentration_array[:, :, [i_timestep], :], gas_boxes, airborne_emissions = step_concentration(
        emissions_array[:, :, [i_timestep], :], 
        gas_boxes,
        airborne_emissions, 
        burden_per_emission_array,
        lifetime_array,
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration_array,
        timestep=1,
        partition_fraction=partition_fraction_array,
        natural_emissions_adjustment=natural_emissions_adjustment_array,
    )

In [ ]:
concentration_array[7, 21, :, 0]

In [ ]:
# def ghg(
#     concentration,
#     pre_industrial_concentration,
#     tropospheric_adjustment,
#     radiative_efficiency,
#     gas_index_mapping,
#     a1 = -2.4785e-07,
#     b1 = 0.00075906,
#     c1 = -0.0021492,
#     d1 = 5.2488,
#     a2 = -0.00034197,
#     b2 = 0.00025455,
#     c2 = -0.00024357,
#     d2 = 0.12173,
#     a3 = -8.9603e-05,
#     b3 = -0.00012462,
#     d3 = 0.045194,
#     ):
    
#     erf_out = np.ones_like(concentration) * np.nan

#     # extracting indices upfront means we're not always searching through array and makes things more readable.
#     # expanding the co2_pi array to the same shape as co2 allows efficient conditional indexing
#     # TODO: what happens if a scenario does not include all these gases?
#     co2 = concentration[:, [gas_index_mapping["CO2"]], ...]
#     co2_pi = pre_industrial_concentration[:, [gas_index_mapping["CO2"]], ...] * np.ones_like(co2)
#     ch4 = concentration[:, [gas_index_mapping["CH4"]], ...]
#     ch4_pi = pre_industrial_concentration[:, [gas_index_mapping["CH4"]], ...]
#     n2o = concentration[:, [gas_index_mapping["N2O"]], ...]
#     n2o_pi = pre_industrial_concentration[:, [gas_index_mapping["N2O"]], ...]

    
#     # CO2
#     ca_max = co2_pi - b1/(2*a1)
#     where_central = np.asarray((co2_pi < co2) & (co2 <= ca_max)).nonzero()
#     where_low = np.asarray((co2 <= co2_pi)).nonzero()
#     where_high = np.asarray((co2 > ca_max)).nonzero()
#     alpha_p = np.ones_like(co2) * np.nan
#     alpha_p[where_central] = d1 + a1*(co2[where_central] - co2_pi[where_central])**2 + b1*(co2[where_central] - co2_pi[where_central])
#     alpha_p[where_low] = d1
#     alpha_p[where_high] = d1 - b1**2/(4*a1)
#     alpha_n2o = c1*np.sqrt(n2o)
#     erf_out[:, [gas_index_mapping["CO2"]], ...] = (alpha_p + alpha_n2o) * np.log(co2/co2_pi) * tropospheric_adjustment[:, [gas_index_mapping["CO2"]], ...]

#     # CH4
#     erf_out[:, [gas_index_mapping["CH4"]], ...] = (
#         (a3*np.sqrt(ch4) + b3*np.sqrt(n2o) + d3) *
#         (np.sqrt(ch4) - np.sqrt(ch4_pi))
#     )

#     # N2O
#     erf_out[:, [gas_index_mapping["N2O"]], ...] = (
#         (a2*np.sqrt(co2) + b2*np.sqrt(n2o) + c2*np.sqrt(ch4) + d2) *
#         (np.sqrt(n2o) - np.sqrt(n2o_pi))
#     )

#     # Then, linear forcing for other gases
#     minor_gas_index = list(range(concentration.shape[SPECIES_AXIS]))
#     for major_gas in ['CO2', 'CH4', 'N2O']:
#         minor_gas_index.remove(gas_index_mapping[major_gas])
#     erf_out[:, minor_gas_index, ...] = (
#         (concentration[:, minor_gas_index, ...] - pre_industrial_concentration[:, minor_gas_index, ...])
#         * radiative_efficiency[:, minor_gas_index, ...] * 0.001
#     ) * tropospheric_adjustment[:, minor_gas_index, ...]
    
#     return erf_out

In [ ]:
erf = ghg(concentration_array, pre_industrial_concentration_array, tropospheric_adjustment_array, radiative_efficiency_array, gas_index_mapping)

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(16,9))
for iscen, scenario in enumerate(scenarios):
    for igas, gas in enumerate(['CO2', 'CH4', 'N2O']):
        ax[igas].plot(np.arange(1750.5, 2501), erf[iscen, gas_index_mapping[gas], :, 0], label='FaIR 2.1')
        ax[igas].set_title(gas)
fig.tight_layout()

In [ ]:
# put back together
effective_radiative_forcing = {}

for iscen, scenario in enumerate(scenarios):
    effective_radiative_forcing[scenario] = {}
    for igas, gas in enumerate(gas_list):
        effective_radiative_forcing[scenario][gas] = erf[iscen, igas, ...]

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    for iscen, scenario in enumerate(scenarios):
        ax[ix, iy].plot(np.arange(1750.5, 2501), effective_radiative_forcing[scenario][gas], label='FaIR 2.1')
    ax[ix, iy].set_title(gas)
fig.tight_layout()